<h1><center>第5章 合并</center></h1>

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('data/table.csv')
df.head()

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


## 一、append与assign
### 1. append方法
#### （a）利用序列添加行（必须指定name）

In [2]:
df_append = df.loc[:3,['Gender','Height']].copy()
df_append

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167


In [3]:
s = pd.Series({'Gender':'F','Height':188},name='new_row')
df_append.append(s)

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167
new_row,F,188


#### （b）用DataFrame添加表

In [4]:
df_temp = pd.DataFrame({'Gender':['F','M'],'Height':[188,176]},index=['new_1','new_2'])
df_append.append(df_temp)

,Gender,Height
0,M,173
1,F,192
2,M,186
3,F,167
new_1,F,188
new_2,M,176


### 2. assign方法
#### 该方法主要用于添加列，列名直接由参数指定：

In [5]:
s = pd.Series(list('abcd'),index=range(4))
df_append.assign(Letter=s)

,Gender,Height,Letter
0,M,173,a
1,F,192,b
2,M,186,c
3,F,167,d


#### 可以一次添加多个列：

In [6]:
df_append.assign(col1=lambda x:x['Gender']*2,
                 col2=s)

,Gender,Height,col1,col2
0,M,173,MM,a
1,F,192,FF,b
2,M,186,MM,c
3,F,167,FF,d


## 二、combine与update
### 1. comine方法
#### comine和update都是用于表的填充函数，可以根据某种规则填充
#### （a）填充对象
#### 可以看出combine方法是按照表的顺序轮流进行逐列循环的，而且自动索引对齐，缺失值为NaN，理解这一点很重要

In [7]:
df_combine_1 = df.loc[:1,['Gender','Height']].copy()
df_combine_2 = df.loc[10:11,['Gender','Height']].copy()
df_combine_1.combine(df_combine_2,lambda x,y:print(x,y))

0       M
1       F
10    NaN
11    NaN
Name: Gender, dtype: object 0     NaN
1     NaN
10      M
11      F
Name: Gender, dtype: object
0     173.0
1     192.0
10      NaN
11      NaN
Name: Height, dtype: float64 0       NaN
1       NaN
10    161.0
11    175.0
Name: Height, dtype: float64


,Gender,Height
0,NaN,NaN
1,NaN,NaN
10,NaN,NaN
11,NaN,NaN


#### （b）一些例子
#### 例①：根据列均值的大小填充

In [8]:
# 例子1
df1 = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
df2 = pd.DataFrame({'A': [8, 7], 'B': [6, 5]})
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y)

,A,B
0,8,6
1,7,5


#### 例②：索引对齐特性（默认状态下，后面的表没有的行列都会设置为NaN）

In [9]:
df2 = pd.DataFrame({'B': [8, 7], 'C': [6, 5]},index=[1,2])
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y)

,A,B,C
0,NaN,NaN,NaN
1,NaN,8.0,6.0
2,NaN,7.0,5.0


#### 例③：使得df1原来符合条件的值不会被覆盖

In [10]:
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y,overwrite=False) 

,A,B,C
0,1.0,NaN,NaN
1,2.0,8.0,6.0
2,NaN,7.0,5.0


#### 例④：在新增匹配df2的元素位置填充-1

In [11]:
df1.combine(df2,lambda x,y:x if x.mean()>y.mean() else y,fill_value=-1)

,A,B,C
0,1.0,-1.0,-1.0
1,2.0,8.0,6.0
2,-1.0,7.0,5.0


#### （c）combine_first方法
#### 这个方法作用是用df2填补df1的缺失值，功能比较简单，但很多时候会比combine更常用，下面举两个例子：

In [12]:
df1 = pd.DataFrame({'A': [None, 0], 'B': [None, 4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
df1.combine_first(df2)

,A,B
0,1.0,3.0
1,0.0,4.0


In [13]:
df1 = pd.DataFrame({'A': [None, 0], 'B': [4, None]})
df2 = pd.DataFrame({'B': [3, 3], 'C': [1, 1]}, index=[1, 2])
df1.combine_first(df2)

,A,B,C
0,NaN,4.0,NaN
1,0.0,3.0,1.0
2,NaN,3.0,1.0


### 2. update方法
#### （a）三个特点
#### ①返回的框索引只会与被调用框的一致（默认使用左连接，下一节会介绍）
#### ②第二个框中的nan元素不会起作用
#### ③没有返回值，直接在df上操作
#### （b）例子
#### 例①：索引完全对齐情况下的操作

In [14]:
df1 = pd.DataFrame({'A': [1, 2, 3],
                    'B': [400, 500, 600]})
df2 = pd.DataFrame({'B': [4, 5, 6],
                    'C': [7, 8, 9]})
df1.update(df2)
df1

,A,B
0,1,4
1,2,5
2,3,6


#### 例②：部分填充

In [15]:
df1 = pd.DataFrame({'A': ['a', 'b', 'c'],
                    'B': ['x', 'y', 'z']})
df2 = pd.DataFrame({'B': ['d', 'e']}, index=[1,2])
df1.update(df2)
df1

,A,B
0,a,x
1,b,d
2,c,e


#### 例③：缺失值不会填充

In [16]:
df1 = pd.DataFrame({'A': [1, 2, 3],
                    'B': [400, 500, 600]})
df2 = pd.DataFrame({'B': [4, np.nan, 6]})
df1.update(df2)
df1

,A,B
0,1,4.0
1,2,500.0
2,3,6.0


## 三、concat方法

## 四、merge与join

## 五、问题与练习